In [1]:
import io
import base64
import requests
import json 
from PIL import Image,ImageDraw
import hashlib
from dotenv import load_dotenv
import re

In [2]:
import os
# Cargar las variables de entorno desde el archivo .env
load_dotenv()
# Acceder a las variables de entorno
ENDPOINT_KEY = os.getenv('ENDPOINT_KEY')

In [3]:
def convert_image(image_path):
    with Image.open(image_path) as img:
        buffer = io.BytesIO()
        if img.format not in ['PNG', 'JPEG', 'GIF', 'WEBP']:
            img.save(buffer, format="JPEG")
            extension = "JPEG"
        else:
            img.save(buffer, format=img.format)
            extension = img.format.lower()
        
        processed_image_path = f"processed_image.{extension}"
            
            
        buffer.seek(0)  
        return buffer.getvalue(),processed_image_path, extension
    

def calculate_checksum(image_data):
    return hashlib.md5(image_data).hexdigest()

In [4]:
def draw_lines_on_image_base64(encoded_image, coordinates):
    image_data = base64.b64decode(encoded_image)
    img = Image.open(io.BytesIO(image_data))
    
    draw = ImageDraw.Draw(img)
    for bottle in coordinates['Image']:
        upper = tuple(bottle['coordinates']['upper'])
        lower = tuple(bottle['coordinates']['lower'])
        draw.line([upper, lower], fill='red', width=8)
    
    result_image_path = "result_image.png"
    img.save(result_image_path)
    return result_image_path

In [5]:
import yaml

class Producto:
    def __init__(self, nombre, cantidad):
        self.nombre = nombre
        self.cantidad = cantidad

    def __repr__(self):
        return f"{self.nombre} ({self.cantidad})"

class Respuesta: 
    def __init__(self, vive100=None, otrosProductos=None):
        self.vive100 = vive100 if vive100 else []
        self.otrosProductos = otrosProductos if otrosProductos else []

    def __repr__(self):
        return f"VIVE100 Products: {self.vive100}\nOther Products: {self.otrosProductos}"

    @classmethod
    def from_yaml(cls, yaml_string):
        # Limpiar el YAML eliminando secuencias de escape y formato adicional
        yaml_string = yaml_string.replace("\\n", "\n").replace("\\u00f3", "ó").replace("\\u00f1", "ñ")
        yaml_string = yaml_string.replace("```yaml", "").replace("```", "").strip()

        # Cargar el YAML
        data = yaml.safe_load(yaml_string)

        vive100_list = []
        otrosProductos_list = []

        if "VIVE100_Products" in data:
            for item in data["VIVE100_Products"]:
                producto = Producto(item["Product"], item["Quantity"])
                vive100_list.append(producto)

        if "Other_Products" in data:
            for item in data["Other_Products"]:
                producto = Producto(item["Product"], item["Quantity"])
                otrosProductos_list.append(producto)

        return cls(vive100=vive100_list, otrosProductos=otrosProductos_list)

In [7]:
# Path to your image file
image_path = "img94.jpg"

# Convert the image
image_data, image_path, extension = convert_image(image_path)

checksum = calculate_checksum(image_data)

print("Checksum:", checksum)

encoded_image = base64.b64encode(image_data).decode('utf-8')
     
endpoint = "https://rume8rx5v6.execute-api.us-east-1.amazonaws.com/pruebas/" 

ENDPOINT_KEY = "dx9XSWUCK56ALu4R35rxZ3z83X2PndhZ1J2ZWnoH"

print(f"Enviando solicitud al servidor...{ENDPOINT_KEY}")
    
headers = {
        "Content-Type": "application/json",
        "x-api-key": ENDPOINT_KEY
        #"x-api-key": "1234567890"
    } 

inputRoot = {
    "Checksum": checksum,
    "Extension": extension,
    "Product" : "vive100",
    "Image": encoded_image
} 

dumbs = json.dumps(inputRoot)

#salvar json
with open('data.json', 'w') as f:
    json.dump(inputRoot, f)

response = requests.post(endpoint, headers=headers, data=dumbs)
    
if response.status_code == 200:
    print("Solicitud recibida del servidor")
    server_response = response.json()
    print("Respuesta del servidor:", server_response)
else:
    print("Error en la solicitud al servidor")
    print(response.text)

Checksum: 7508a01d2c36c938779dd11211706890


Enviando solicitud al servidor...dx9XSWUCK56ALu4R35rxZ3z83X2PndhZ1J2ZWnoH
Solicitud recibida del servidor
Respuesta del servidor: {'body': {'una excepcion lanzada': 'An error occurred (AllAccessDisabled) when calling the PutObject operation: All access to this object has been disabled'}, 'statusCode': 500}


In [12]:

yaml_string = server_response['body'].strip('"')


print("body yaml result: "+yaml_string)

respuesta = Respuesta.from_yaml(yaml_string)

# Imprimir la respuesta formateada
print(respuesta)

body yaml result: ```yaml\nVIVE100_Products:\n  - Product: vive100 verde original\n    Quantity: 2\n  - Product: vive100 morado azai\n    Quantity: 1\n  - Product: vive100 rojo normal\n    Quantity: 1\n\nOther_Products:\n  - Product: Hit Lulo\n    Color: verde\n    Quantity: 3\n  - Product: Natural Maracuya\n    Color: amarillo\n    Quantity: 2\n```
VIVE100 Products: [vive100 verde original (2), vive100 morado azai (1), vive100 rojo normal (1)]
Other Products: [Hit Lulo (3), Natural Maracuya (2)]
